In [ ]:
# Install necessary packages (run in a Colab cell)
!pip install numpy==1.24.4 gensim matplotlib seaborn --quiet
!pip install gnews --quiet

In [ ]:
from gnews import GNews
import pandas as pd

# Initialize GNews
gnews = GNews(language='en', country='IN', max_results=100)

# Fetch articles on "climate change"
articles = gnews.get_news('climate change')

# Convert to DataFrame
df_news = pd.DataFrame(articles)
df_news = df_news[['title', 'description', 'published date', 'url']]
df_news.dropna(subset=['description'], inplace=True)
df_news.reset_index(drop=True, inplace=True)

# Preview data
df_news.head()


,title,description,published date,url
0,High-income groups disproportionately contribu...,High-income groups disproportionately contribu...,"Wed, 07 May 2025 09:53:26 GMT",https://news.google.com/rss/articles/CBMiX0FVX...
1,Why climate change is a big threat to your eye...,Why climate change is a big threat to your eye...,"Wed, 07 May 2025 10:40:07 GMT",https://news.google.com/rss/articles/CBMizwFBV...
2,Climate change on your doorstep: Online tool s...,Climate change on your doorstep: Online tool s...,"Tue, 06 May 2025 18:19:04 GMT",https://news.google.com/rss/articles/CBMiekFVX...
3,"Climate change a global challenge, need to pro...","Climate change a global challenge, need to pro...","Mon, 05 May 2025 21:42:36 GMT",https://news.google.com/rss/articles/CBMiqgFBV...
4,Opinion | Climate Crisis and The Threat of Irr...,Opinion | Climate Crisis and The Threat of Irr...,"Tue, 06 May 2025 12:50:16 GMT",https://news.google.com/rss/articles/CBMikwFBV...


In [ ]:
import re

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W+', ' ', text)  # remove non-word characters
    text = re.sub(r'\d+', '', text)   # remove numbers
    return text.strip()

df_news['cleaned_text'] = df_news['description'].apply(preprocess_text)
texts = [text.split() for text in df_news['cleaned_text']]


In [ ]:
from gensim import corpora, models

# Dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA model
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    passes=10,
    random_state=42
)

# Print topics
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx + 1}: {topic}")


Topic 1: 0.061*"climate" + 0.048*"change" + 0.029*"the" + 0.025*"of" + 0.021*"and" + 0.019*"to" + 0.017*"gov" + 0.013*"in" + 0.013*"science" + 0.012*"s"
Topic 2: 0.047*"climate" + 0.043*"the" + 0.035*"of" + 0.034*"change" + 0.022*"world" + 0.021*"global" + 0.018*"warming" + 0.016*"to" + 0.015*"s" + 0.012*"in"
Topic 3: 0.014*"an" + 0.014*"one" + 0.008*"the" + 0.008*"will" + 0.008*"geoengineering" + 0.008*"why" + 0.008*"unprecedented" + 0.008*"a" + 0.008*"live" + 0.008*"greenpeace"
Topic 4: 0.034*"climate" + 0.032*"and" + 0.030*"change" + 0.019*"of" + 0.016*"how" + 0.016*"on" + 0.014*"the" + 0.013*"earth" + 0.013*"out" + 0.009*"action"
Topic 5: 0.054*"climate" + 0.038*"change" + 0.015*"to" + 0.013*"and" + 0.013*"how" + 0.013*"com" + 0.011*"s" + 0.011*"the" + 0.010*"org" + 0.010*"of"


In [ ]:
def get_dominant_topic(text):
    bow = dictionary.doc2bow(text.split())
    topic_probs = lda_model.get_document_topics(bow)
    return sorted(topic_probs, key=lambda x: x[1], reverse=True)[0][0]

df_news['topic_id'] = df_news['cleaned_text'].apply(get_dominant_topic)

# Manual topic labels (adjust based on print_topics result)
topic_names = {
    0: "Carbon Emissions",
    1: "Extreme Weather",
    2: "Global Warming",
    3: "Agricultural Impact",
    4: "Sea Level Rise"
}
df_news['topic_label'] = df_news['topic_id'].map(topic_names)


In [ ]:
from collections import Counter

def generate_insight(topic_id):
    insights = {
        0: "High carbon emissions are driving global warming. Reducing emissions is critical.",
        1: "Extreme weather events are intensifying due to climate change.",
        2: "Rising global temperatures result from unchecked greenhouse gas emissions.",
        3: "Agriculture is increasingly impacted by climate shifts like drought and heat.",
        4: "Melting ice caps and rising seas threaten coastal regions."
    }
    return insights.get(topic_id, "Insight not available.")

def suggest_policies(topic_id):
    return {
        0: ["Carbon taxes", "Renewable energy incentives", "Emission caps for industries"],
        1: ["Disaster-resilient infrastructure", "Early warning systems", "Better urban planning"],
        2: ["Global carbon agreements", "Forest conservation", "Green tech funding"],
        3: ["Climate-resilient crops", "Efficient irrigation", "Subsidies for sustainable farming"],
        4: ["Coastal protection", "Development zoning", "Managed retreat strategies"]
    }.get(topic_id, ["No policy suggestions available."])


In [ ]:
def get_topic_by_user_input(user_input):
    topic_mapping = {
        "carbon emissions": 0,
        "extreme weather": 1,
        "global warming": 2,
        "agricultural impact": 3,
        "sea level rise": 4
    }
    return topic_mapping.get(user_input.lower(), None)

def display_descriptive_output(user_input):
    topic_id = get_topic_by_user_input(user_input)

    if topic_id is None:
        print("❌ The topic you entered is not recognized. Please choose from one of the predefined climate issues.")
        return

    topic_title = user_input.title()
    insight = generate_insight(topic_id)
    policies = suggest_policies(topic_id)

    print("=" * 80)
    print(f"🌍 Climate Issue Selected: **{topic_title}**\n")
    print("🧠 Insight Summary:")
    print(f"{insight}\n")

    print("📌 Context from Recent News:")
    related_articles = df_news[df_news['topic_id'] == topic_id].head(3)
    for i, row in related_articles.iterrows():
        print(f"• \"{row['title']}\"")
        print(f"  Description: {row['description']}")
        print(f"  Link: {row['url']}\n")

    print("📜 Recommended Policy Actions:")
    for idx, policy in enumerate(policies, 1):
        print(f"{idx}. {policy}")

    print("\n✅ These recommendations are inferred from current climate discourse trends and may assist policymakers in targeting the root causes effectively.")
    print("=" * 80)

# Call the function
print("Available Topics:\n- Carbon Emissions\n- Extreme Weather\n- Global Warming\n- Agricultural Impact\n- Sea Level Rise\n")
user_input = input("Enter the climate issue you want insights on: ")
display_descriptive_output(user_input)


Available Topics:
- Carbon Emissions
- Extreme Weather
- Global Warming
- Agricultural Impact
- Sea Level Rise

Enter the climate issue you want insights on: Global Warming
🌍 Climate Issue Selected: **Global Warming**

🧠 Insight Summary:
Rising global temperatures result from unchecked greenhouse gas emissions.

📌 Context from Recent News:
• "Will you live an unprecedented life? - EurekAlert!"
  Description: Will you live an unprecedented life?  EurekAlert!
  Link: https://news.google.com/rss/articles/CBMiXEFVX3lxTFB5M00tZy1iT1J2RmNGTU1JX1oyNFVUc3ZRTnBHXzVpdVVZTkU5M29xak12VEZIdHB5WmZHT1ZrbHNlZ1RhQXhKamg4R2JUOHBRSWU5T19OYzRTOWJ3?oc=5&hl=en-US&gl=US&ceid=US:en

• "Could conserving tigers in forests be an integral part of climate change solutions? - Phys.org"
  Description: Could conserving tigers in forests be an integral part of climate change solutions?  Phys.org
  Link: https://news.google.com/rss/articles/CBMidkFVX3lxTE9FMENuM0dNbnNjYTJSRWsxVHhjOE5wblFCclNyTVZ6WXBIZzZXSnM3cDEwUXNWMUk

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr

def climate_insight_tool(user_topic):
    topic_id = get_topic_by_user_input(user_topic)

    if topic_id is None:
        return (
            "❌ Topic not recognized. Please choose a valid one.",
            "", "", ""
        )

    topic_title = user_topic.title()
    insight = generate_insight(topic_id)
    policies = suggest_policies(topic_id)

    # Get top 3 related articles
    related_articles = df_news[df_news['topic_id'] == topic_id].head(3)
    article_output = ""
    for _, row in related_articles.iterrows():
        article_output += f"• {row['title']}\n  {row['description']}\n  [Read More]({row['url']})\n\n"

    policies_output = "\n".join([f"{i+1}. {policy}" for i, policy in enumerate(policies)])

    return (
        f"🧠 **Insight on {topic_title}**:\n{insight}",
        f"📰 **Recent News Articles**:\n\n{article_output}",
        f"📜 **Policy Recommendations**:\n\n{policies_output}",
        "✅ These recommendations are based on topic modeling from live climate-related news data."
    )

# Launch the Gradio app
topics = ["Carbon Emissions", "Extreme Weather", "Global Warming", "Agricultural Impact", "Sea Level Rise"]

gr.Interface(
    fn=climate_insight_tool,
    inputs=gr.Dropdown(choices=topics, label="Select Climate Issue"),
    outputs=[
        gr.Markdown(label="Insight"),
        gr.Markdown(label="Recent News"),
        gr.Markdown(label="Policy Suggestions"),
        gr.Markdown(label="Conclusion")
    ],
    title="🌍 EcoNLP: Climate Change Insight Assistant",
    description="Get real-time insights and policy suggestions based on environmental news data using topic modeling."
).launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2785d9aa3ba163b14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
